# Домашнее задание №5
## Решение СЛАУ при помощи LU и LUP разложений
---
**Выполнила:** Халфина Айсылу Зуфаровна

**Группа:** НПМбд-02-19

---

### Цели работы:
- знакомство с библиотекой numpy;
- реализация методов LU и LUP разложений для решения систем линейных алгебраических уравнений (СЛАУ);
- сравнительный анализ эффективности методов для решения СЛАУ.

Перед началом работы импортируем необходимые библиотеки.

In [7]:
import numpy as np
import scipy.linalg as sl

# LU-разложение

$LU-разложение$ (LU-декомпозиция, LU-факторизация) — представление матрицы $A$ в виде произведения двух матриц, $A=LU$, где $L$ — нижняя треугольная матрица, а $U$ — верхняя треугольная матрица.

$LU-разложение$ используется для решения систем линейных уравнений, обращения матриц и вычисления определителя. $LU-разложение$ существует только в том случае, когда матрица $A$ обратима, а все ведущие (угловые) главные миноры матрицы $A$ невырождены.

Этот метод является одной из разновидностей метода Гаусса.

Первым делом реализуем функцию, принимающую на вход оригинальную квадратную матрицу и возвращающую $L$ и $U$ матрицы.

In [9]:
def LU(a):
    rows = a.shape[0]
    u = a.copy()
    l = np.eye(rows)
    for i in range(rows):
        f = u[i+1:, i] / u[i, i]
        l[i+1:, i] = f
        u[i+1:] -= f[:, np.newaxis] * u[i]
    return l, u

Пусть дана СЛАУ вида $A x = b$. С помощью LU-разложения её можно свести к $L U x = b$.

Эта система может быть решена в два шага:

1. Решается система $L y = b$, которая решается прямой подстановкой, т.к. $L$ --- нижняя треугольная матрица.

2. Решается система $U x = y$, которая решается обратной подстановкой, т.к. $U$ --- верхняя треугольная матрица.

Реализуем функцию, находящую решение СЛАУ при помощи LU-разложения.

In [10]:
def solve_LU(A,b):
    if A.shape[0] != A.shape[1]:
        return 'Матрица не квадратная'
    if np.linalg.det(A) == 0:
        return 'Матрица вырождена'
    N = len(A)
    for i in range(1, N):
        if np.linalg.det(A[:i, :i]) == 0:
            return 'Главный минор вырожден'
    L, U = LU(A)
    y = np.zeros(N)
    x = np.zeros(N)
    for i in range(N):
        y[i] = b[i] - L[i].dot(y)
    for i in range(N - 1, -1, -1):
        x[i] = (y[i] - U[i].dot(x)) / U[i, i]
    return x             

# LUP-разложение

$LUP-разложение$ (LUP-декомпозиция) — представление данной матрицы $A$ в виде произведения $PA = LU$ где матрица $L$ является нижнетреугольной с единицами на главной диагонали, $U$ — верхнетреугольная общего вида, а $P$ — т. н. матрица перестановок, получаемая из единичной матрицы путём перестановки строк или столбцов. Такое разложение можно осуществить для любой невырожденной матрицы. $LUP-разложение$ используется для вычисления обратной матрицы по компактной схеме, вычисления решения системы линейных уравнений. По сравнению с алгоритмом $LU-разложения$ алгоритм $LUP-разложения$ может обрабатывать любые невырожденные матрицы и при этом обладает более высокой вычислительной устойчивостью.

Сперва реализуем функцию, которая будет менять местами строки матрицы.

In [12]:
def switch(A,i,j):
    temp = np.copy(A[i])
    A[i] = A[j]
    A[j] = temp
    return A

Затем реализуем функцию, находящую LUP-разложение матрицы $А$.

In [13]:
def LUP(A):
    N = len(A)
    P = np.identity(N)
    E = np.identity(N)
    B = np.copy(A)
    for i in range(N - 1):
        max_ind = np.argmax(abs(B[:, i]))
        if i != max_ind:
            B = switch(B, i, max_ind)
            E = switch(E, i, max_ind)
            P = switch(P, i, max_ind)
        mult = np.array(B[(i + 1):,i] / (- B[i, i]))
        mult = np.array([mult,]).T
        B_add = mult.dot([B[i],])
        E_add = mult.dot([E[i],])
        B[(i + 1):] += B_add
        E[(i + 1):] += E_add
    U = np.copy(B)
    L = P.dot(np.linalg.inv(E))
    return L, U, P

Пусть дана СЛАУ вида $A x = b$. С помощью LUP-разложения, домножив левую и правую части на $P$, её можно свести к $L U x = P b$.

Эта система может быть решена в два шага:

1. Решается система $L y = P b$, которая решается прямой подстановкой, т.к. $L$ --- нижняя треугольная матрица.

2. Решается система $U x = y$, которая решается обратной подстановкой, т.к. $U$ --- верхняя треугольная матрица.

Реализуем функцию, находящую решение СЛАУ при помощи LUP-разложения.

In [14]:
def solve_LUP(A, b):
    if A.shape[0] != A.shape[1]:
        return 'Матрица не квадратная'
    if np.linalg.det(A) == 0:
        return 'Матрица вырождена'
    N = len(A)
    L, U, P = LUP(A)
    b = P.dot(b)
    y = np.zeros(N)
    x = np.zeros(N)
    for i in range(N):
        y[i] = b[i] - L[i].dot(y)
    for i in range(N - 1, -1, -1):
        x[i] = (y[i] - U[i].dot(x)) / U[i, i]
    return x   

# Проверка решений

Протестируем решение СЛАУ при помощи функции LU разложения.

In [19]:
Matrix = np.array([[1.,2,1],[2,1,1],[1,-1,2]])
b = [0.,1,0]

Answer = solve_LU(Matrix,b)
print('Ответ:')
print(Answer)

Ответ:
[ 0.83333333 -0.16666667 -0.5       ]


Теперь протестируем решение при помощи функции LUP разложения

In [20]:
Matrix = np.array([[1.,2,1],[2,1,1],[1,-1,2]])
b = [0.,1,0]

Answer = solve_LUP(Matrix,b)
print('Ответ:')
print(Answer)

Ответ:
[ 0.83333333 -0.16666667 -0.5       ]


Результаты работы обеих функций совпали. Теперь проверим их при помощи библиотеки scipy.

In [21]:
Matrix = np.array([[1.,2,1],[2,1,1],[1,-1,2]])
b = [0.,1,0]

Answer = sl.solve(Matrix,b)
print('Ответ:')
print(Answer)

Ответ:
[ 0.83333333 -0.16666667 -0.5       ]


# Вывод

Оба алгоритма отработали корректно и выдали верный результат. Однако LU-разложение для произвольной квадратной невырожденной матрицы возможно не всегда, необходимо, чтобы в процессе решения не было деления на ноль, в противном случае эффективнее использовать LUP-разложение, отличие которого в том, что перед каждым шагом нахождения коэффициентов, необходимых для последующего сложения строк, присутствует операция перестановки местами двух строк соответствующей матрицы (с выбором наибольшего по модулю ведущего элемента), обеспечивающая получение ненулевых ведущих элементов.